# LIONS Analysis

Our project is to analyze federal cases.

In [1]:
from lions.parse import *
import pandas as pd
import numpy as np

In [2]:

case_sql, cases = gs_case()
hist_sql, hist = gs_court_hist()
print(cases[:5])
hist[:5]

[{'DISTRICT': 'AK ', 'ID': '1983V01272', 'CLASS': 'V', 'NAME': '*                                                 ', 'RECVD_DATE': '06-JUL-1983', 'SECURITY': ' ', 'PRIORITY': ' ', 'TYPE': '    ', 'US_ROLE': 'D ', 'LIT_RESP': 'DJ', 'LIT_TRACK': ' ', 'WEIGHT': '  ', 'BRANCH': '1  ', 'GRAND_JURY_NUM': '               ', 'UNIT': '    ', 'DCMNS_NUMBER': '97-627063-0', 'TRIBE': '    ', 'RESERVATION': '    ', 'SPEC_PROJ': '  ', 'VIC_WIT': ' ', 'ADR_MODE': '  ', 'COLLECT_IND': 'N', 'OFFENSE_FROM': '           ', 'OFFENSE_TO': '           ', 'LEAD_CHARGE': '                         ', 'PHYSICAL_LOC': '                    ', 'STORE_NUM': '                    ', 'CIVIL_POTEN': ' ', 'SYS_INIT_DATE': '31-AUG-1991', 'ACCESS_DATE': '15-FEB-2000', 'STATUS': 'C', 'CLOSE_DATE': '15-FEB-2000', 'DEST_DATE': '12-FEB-2010', 'PERMANENT': 'N', 'CASE_RESTRICTED': ' ', 'CRIMINAL_POTEN': ' ', 'TOT_VICTIMS': '           ', 'RELATED_FLU_FLAG': ' ', 'QUI_TAM': ' ', 'CREATE_DATE': '17-NOV-2006', 'CREATE_USER': '', '

[{'APPEAL_TYPE': ' ',
  'CASEID': '1983V01272',
  'COURT': 'DC',
  'COURT_NUMBER': '*                        ',
  'CREATE_DATE': '           ',
  'CREATE_USER': '                              ',
  'DISPOSITION': 'JX',
  'DISP_DATE': '04-NOV-1997',
  'DISP_REASON1': '    ',
  'DISP_REASON2': '    ',
  'DISP_REASON3': '    ',
  'DISTRICT': 'AK        ',
  'FILING_DATE': '06-JUL-1983',
  'ID': '1         ',
  'LOCATION': '  ',
  'NOAP_DATE': '           ',
  'SENT_APPEAL': ' ',
  'SERVICE_DATE': '           ',
  'SYS_DISP_DATE': '15-FEB-2000',
  'SYS_FILING_DATE': '06-JUL-1983',
  'TRIAL_DAYS': '0             ',
  'UPDATE_DATE': '           ',
  'UPDATE_USER': '                              ',
  'US_ROLE': 'D '},
 {'APPEAL_TYPE': ' ',
  'CASEID': '1984V01251',
  'COURT': 'DC',
  'COURT_NUMBER': '*                        ',
  'CREATE_DATE': '           ',
  'CREATE_USER': '                              ',
  'DISPOSITION': 'JU',
  'DISP_DATE': '04-NOV-1997',
  'DISP_REASON1': '    ',
  'DIS

In [ ]:
test_cases = cases[:2000]
test_hist = hist[:2000]
cases_df = pd.DataFrame(test_cases, columns = test_cases[0].keys())
hist_df = pd.DataFrame(test_hist, columns = test_hist[0].keys())

print(hist_df['DISPOSITION'])
print(len(cases))
print(len(hist))

case_hist_df = pd.merge(cases_df, hist_df, left_on = 'ID', right_on = 'CASEID', how = 'inner')

print(len(case_hist_df))
withdisp_df = case_hist_df[case_hist_df.DISPOSITION != '  ']
print(len(withdisp_df))
withdisp_reason_df = withdisp_df[withdisp_df.DISP_REASON1 != '    ']
print(len(withdisp_reason_df))

0          JX
1          JU
2          LN
3          JU
4          DJ
5          JU
6          GT
7          GT
8          CW
9          JU
10         JU
11         FA
12         DM
13         FA
14         FA
15         GT
16         GT
17         DM
18         DM
19         JU
20         DT
21         DH
22         NW
23         DM
24         BB
25         DC
26         DH
27         DW
28         JT
29         DR
           ..
8262321      
8262322      
8262323      
8262324    JX
8262325      
8262326    JU
8262327      
8262328      
8262329    JU
8262330      
8262331      
8262332    CW
8262333      
8262334      
8262335      
8262336      
8262337      
8262338      
8262339      
8262340    OE
8262341      
8262342      
8262343      
8262344      
8262345      
8262346      
8262347      
8262348      
8262349      
8262350      
Name: DISPOSITION, Length: 8262351, dtype: object
6041748
8262351
